<div style="text-align: center;">

  <img src="img/LogoUesc.png" alt="UESC" style="width: 60px; display: block; margin: 0 auto 5px auto;">

  <div style="font-size: 20px;"><strong>UNIVERSIDADE ESTADUAL DE SANTA CRUZ - UESC</strong></div>
  <div style="font-size: 18px;">DEPARTAMENTO DE ENGENHARIAS E COMPUTAÇÃO - DEC</div>
  <div style="font-size: 18px;">Grupo de Pesquisas em Simulações e Controle de Processos</div>

  <hr style="margin: 20px 0; border: 1px solid #0074B7;">

  <div style="font-size: 22px; font-weight: bold; margin-bottom: 10px;">PROGRAMAÇÃO EM PYTHON PARA AS ENGENHARIAS</div>

  <div style="font-size: 18px; margin-top: 10px;"><strong>Tema</strong>: Previsão da Composição em uma Corrente de Destilação com Redes Neurais</div>
  <div style="font-size: 18px;"><strong>Prof. Dr. E.R.Edwards</strong></div>

  <div style="font-size: 16px; margin-top: 15px;">Ilhéus - BA, Abril de 2025</div>
  
  <hr style="margin: 20px 0; border: 1px solid #0074B7;">

</div>

__Exercício – Previsão da Composição em uma Corrente de Destilação com Redes Neurais__.

__Enunciado__:

Uma planta de destilação de etanol coleta dados operacionais de várias correntes do processo para controlar a qualidade do produto. Os engenheiros estão interessados em prever a fração mássica de etanol em diferentes correntes, com base em variáveis de processo como temperatura e vazão mássica.

A tabela abaixo mostra dados de 4 correntes retiradas da coluna de destilação:

<img src="img/Tab_06.png" alt="UESC" style="width: 550px; display: block; margin: 0 auto 5px auto;">
<p style="text-align: center;"><strong>Figura: Tabela de Coleta de Dados em uma Planta de Destilação de etanol.</strong></p>

__Objetivo__:

Utilize uma __rede neural__ simples para prever a __fração de etanol__ com base na __temperatura__ e na __vazão__.

__Instruções__:
1. Construa uma rede neural do zero (sem bibliotecas prontas), com:
- Duas entradas: temperatura e vazão
- Uma camada oculta com neurônios à sua escolha
- Uma saída: fração de etanol (normalizada entre 0 e 1)

2. Normalize os dados de entrada e saída, se necessário.
   
3. Faça previsões para novas correntes, por exemplo:

<img src="img/Tab_07.png" alt="UESC" style="width: 300px; display: block; margin: 0 auto 5px auto;">
<p style="text-align: center;"><strong>Figura: Tabela de dados para previsão de novas correntes.</strong></p>

4. Interprete os resultados: a rede neural consegue generalizar a partir dos dados fornecidos?

__Implementada do zero, sem bibliotecas prontas como Keras.__

1. __Dados brutos__
2. __Normalização dos dados (mín-máx)__

In [1]:
import numpy as np

# Função de normalização mín-máx
def normalizar(dados):
    minimo = dados.min(axis=0)
    maximo = dados.max(axis=0)
    return (dados - minimo) / (maximo - minimo), minimo, maximo

# Dados de entrada: Temperatura (°C), Vazão (kg/h)
entradas = np.array([
    [78, 1200],  # Corrente A
    [72, 1350],  # Corrente B
    [80, 1100],  # Corrente C
    [70, 1250],  # Corrente D
], dtype=np.float32)

# Dados de saída: Fração de etanol (%)
saidas = np.array([
    [95],
    [88],
    [97],
    [85],
], dtype=np.float32)

# Normalizando entradas e saídas
entradas_norm, entradas_min, entradas_max = normalizar(entradas)
saidas_norm, saidas_min, saidas_max = normalizar(saidas)


3. Funções de ativação e suas derivadas.

In [2]:
# Função ReLU e derivada
def relu(x):
    return np.maximum(0, x)

def relu_derivada(x):
    return (x> 0).astype(float)    

4. Inicialização da rede neural

In [3]:
# Inicialização dos pesos.
np.random.seed(42)

# 2 entradas, 3 neurônios na camada oculta, 1 saída.
pesos_entrada_oculta = np.random.randn(2,3)
bias_oculta = np.zeros((1,3))

pesos_oculta_saida = np.random.randn(3, 1)
bias_saida = np.zeros((1,1))

#Hiperparâmetros
taxa_aprendizado = 0.1
epochs = 1000

5. Treinamento da rede neural

In [4]:
for epoca in range(epochs):
    # FORWARD
    camada_oculta_input = np.dot(entradas_norm, pesos_entrada_oculta) + bias_oculta
    camada_oculta_saida = relu(camada_oculta_input)

    saida_input = np.dot(camada_oculta_saida, pesos_oculta_saida) + bias_saida
    saida_predita = saida_input  # Sem função de ativação final (regressão)

    # ERRO
    erro = saidas_norm - saida_predita

    # BACKPROPAGATION
    d_saida = -2 * erro  # derivada MSE

    d_pesos_oculta_saida = np.dot(camada_oculta_saida.T, d_saida)
    d_bias_saida = d_saida.sum(axis=0, keepdims=True)

    d_oculta = np.dot(d_saida, pesos_oculta_saida.T) * relu_derivada(camada_oculta_input)
    d_pesos_entrada_oculta = np.dot(entradas_norm.T, d_oculta)
    d_bias_oculta = d_oculta.sum(axis=0, keepdims=True)

    # ATUALIZAÇÃO
    pesos_oculta_saida -= taxa_aprendizado * d_pesos_oculta_saida
    bias_saida -= taxa_aprendizado * d_bias_saida

    pesos_entrada_oculta -= taxa_aprendizado * d_pesos_entrada_oculta
    bias_oculta -= taxa_aprendizado * d_bias_oculta

    # Exibir erro médio a cada 200 épocas
    if epoca % 200 == 0:
        mse = np.mean(erro**2)
        print(f'Época {epoca}, Erro quadrático médio: {mse:.4f}')

Época 0, Erro quadrático médio: 2.0021
Época 200, Erro quadrático médio: 0.1680
Época 400, Erro quadrático médio: 0.1680
Época 600, Erro quadrático médio: 0.1680
Época 800, Erro quadrático médio: 0.1680


6. Previsão com novos dados

In [5]:
# Novas entradas
novas_entradas = np.array([
    [75, 1300],
    [73, 1180]
], dtype=np.float32)

# Normalizar novas entradas com base nos dados originais
novas_entradas_norm = (novas_entradas - entradas_min) / (entradas_max - entradas_min)

# Forward para previsão
camada_oculta_nova = relu(np.dot(novas_entradas_norm, pesos_entrada_oculta) + bias_oculta)
saida_nova_norm = np.dot(camada_oculta_nova, pesos_oculta_saida) + bias_saida

# Desnormalizar saída
saida_nova = saida_nova_norm * (saidas_max - saidas_min) + saidas_min
print("\nPrevisões para novas correntes:")
for i, pred in enumerate(saida_nova):
    print(f"Corrente {i+1}: Fração de etanol estimada = {pred[0]:.2f}%")



Previsões para novas correntes:
Corrente 1: Fração de etanol estimada = 91.25%
Corrente 2: Fração de etanol estimada = 91.25%


__Conclusão sobre os resultados da Rede Neural__.

A rede neural foi treinada com quatro amostras de dados referentes a correntes de uma coluna de destilação, utilizando temperatura e vazão mássica como entradas para prever a fração de etanol (%).

Após o treinamento, a rede foi utilizada para prever a fração de etanol em duas novas correntes com condições intermediárias. O modelo previu aproximadamente:

- Corrente 1: $91,25 \%$
- Corrente 2: $91,25 \%$

__Conclusão didática__:

O modelo treinado conseguiu aprender uma relação coerente entre variáveis operacionais (temperatura e vazão) e a fração de etanol. Mesmo com uma base de dados pequena, ele foi capaz de prever valores razoáveis para novas condições. Isso demonstra o potencial das redes neurais como ferramenta de modelagem e previsão de variáveis de processo na Engenharia Química.

<div style="text-align: center; font-size: 12px; color: gray; margin-top: 40px;">
  Este notebook foi desenvolvido no âmbito do Grupo de Pesquisas em Modelagem Computacional da UESC.<br>
  Todos os direitos reservados © 2025
</div>